In [ ]:
from json import load as jsonload
from functools import wraps, cache
from collections.abc import Iterable # Sized,

In [ ]:
def iterable_info(iterable):
    # important outcomes:
        # tuple version
        # original format
            # is_str
            # is_nonstr_iter
            # is_noniter
        # length
            # is_size_1
    if isinstance(iterable, str):
        return ((iterable,), 2, 1)
    if isinstance(iterable, Iterable):
        # if not isinstance(iterable, tuple):
        #     iterable = tuple(iterable)
        # return (iterable, 1, len(iterable))
        return (tuple(iterable), 1, len(iterable))
    return ((iterable,), 0, 1)



def invert(dictionary):
    if type(dictionary) is not dict:
        dictionary = dict(enumerate(dictionary))
    inverse = {}
    for i, j in dictionary.items():
        if iterable_info(j)[1] != 1:
            j = (j,)
        for k in j:
            inverse.setdefault(k, [])
            inverse[k].append(i)
    # print(inverse)
    return inverse


def pack_func(func, *args, **kwargs):
    return tuple(func, args, kwargs)


def format_index(index, tuple_size = None, default = None):
    if index is None:
        if default is None:
            raise ValueError()
        return default
    info = iterable_info(index)
    if info[1] == 1:
        if info[2] == 1:
            index = index[0]
        elif info[2] == tuple_size:
            return index
        else:
            raise ValueError()
    if tuple_size is None:
        return index
    return tuple([index]*(tuple_size))


# def both(func, *args):
#     return func(*args), *args


# decorator factory for manual debugging
def test_return(preprint_list = None, prelineprint = None, postlineprint = None, postprint_list = None):
    if preprint_list is None:
        preprint_list = []
    if postprint_list is None:
        postprint_list = []
    if prelineprint is None:
        prelineprint = ''
    if postlineprint is None:
        postlineprint = ''

    def decorator(func):

        @wraps(func)
        def wrapper(*args, **kwargs):
            retval = func(*args, **kwargs)
            for m in preprint_list:
                print(m)
            print(f"{prelineprint}{retval}{postlineprint}")
            for m in postprint_list:
                print(m)
            return retval
        
        return wrapper
    
    return decorator
    

# cache decorator that allows caching exceptions too
def cache_with_except(func):
        
    @wraps(func)
    @cache
    def inner_wrapper(*inner_args, **inner_kwargs):
        try:
            inner_retval = func(*inner_args, **inner_kwargs)
        except Exception as e:
            return e
        return inner_retval

    @wraps(func)
    def outer_wrapper(*outer_args, **outer_kwargs):
        outer_retval = inner_wrapper(*outer_args, **outer_kwargs)
        if isinstance(outer_retval, Exception):
            raise outer_retval
        else:
            return outer_retval
    
    return outer_wrapper

In [ ]:
class SKDescriptors:
    ### This class is separated into 7 sections, separated by long rows of asterisks. These sections are:
        # 1. File Specifiers -- These describe key details of collected data and its labels
        # 2. Short Descriptions -- These are values or dictionaries that describe certain file specifier values (e.g. num of classes in Type 5)
        # 3. Tags and Class Names -- This lists all currently used tags and classes
        # 4. ClassType Definitions -- This section defines ClassType and lists each classtype's tags and the relationships it has with other classtypes
        # 5. LabelTypes' ClassTypes -- This section specifically lists which classtypes each label type has
#   *** # 6. Supporting Functions -- This section holds most of the actual helper functions
        # 7. Validating Functions -- This section holds all the validators that this class provides





    # 1. File Specifiers *****************************************************************************************************************************



    # file specification list:
    # NOTE: These should appear in listed order for the code to work
        # (though the code will run well even if some are missing)
    # NOTE: 0 should (almost) always be treated as the unknown value.
        # if you add something that uses the value 0, consider what
        # might need changed in the code to make it work
        # NOTE: One of my next steps will be to made the unknown value 'X' instead
    # NOTE: I have plans to add an InputType / *DatType* (DataType) / ReadType
    # NOTE: I have plans to change Type to LabelType and will need to allow for backwards compatability
        # allowing the code to still successfully read Type but

        # (1) Type (ClassificationType, *LabelType,* OutputType):
            # This specifies how the data is stored.
            # USAGE: This appears ONLY in Labeled specifier lists
            # INFO: The current types' definitions can be found in the files
                # DataTranslations.docx
                # DataCollectionAndLabelingTechniquesDocumentation.docx

        # (2) WithClassNum: this specifies how many output classes a type has
            # so one immediately knows without having to look it up.
            # USAGE: This appears ONLY in Labeled specifier lists
            # NOTE: WithClassNum should not change separately to Type.
                # This is just here to better convey classification info

        # (3) Freq: this specifies the frequency the data collector
            # was set to when collecting that set of data.
            # USAGE: This appears BOTH in Labeled and Unlabeled specifier lists
            # NOTE: there are currently no functions to convert between frequencies
                # as we have only used one frequency so far,
                # and as we will have to test whether the values change with
                # frequency in some way due to how the measurements are taken
            # NOTE: if you ever change the Freq of the file, be sure to adjust
                # BufferNum accordingly

        # (4) BufferType: this specifies how and where buffers are added to labeled data.
            # USAGE: This appears ONLY in Labeled specifier lists
            # NOTE: there are currently no functions to convert between buffer types
                # as there is only one buffer type.

        # (5) BufferNum: this specifies how many measurements before and after
            # certain activities should be labeled the same as that activity
            # USAGE: This appears ONLY in Labeled specifier lists

        # (6) UserType: this states who collected the data --
            # us researchers (marked as 1) or manual wheelchair users (marked as 2).
            # Data from researchers who are also manual wheelchair users should be
            # marked as 3. Data combined from files marked with ((1 and/or 3) AND 2)
            # should be marked as 4 (combining less-biased data with more-biased data)
            # USAGE: This appears BOTH in Labeled and Unlabeled specifier lists
            # NOTE: Unmarked files are automatically labeled with a 0 when passing
                # through the converter and must be manually remarked. If you are
                # unsure which mark is correct, leave it as 0 and treat it as a 1
                # when using the data

        # (7) UserID: This allows us to loosely tell which data files were made by the
            # same person. The UserID may be allowed to change occasionally if keeping
            # it the same might be unfeasible. UserID of 0 means unknown.
            # USAGE: This appears BOTH in Labeled and Unlabeled specifier lists.
            # INFO: Researchers' UserIDs will be stored in [not-yet-created-document name]
            # NOTE: UserID should NOT be stored alongside and with correspondence to
                # the user's personal information unless Dr. Fu (or other person heading
                # the project if that changes) says otherwise as that may have legal
                # implications. In other words, data we store should NOT be directly
                # traceable back to the user who created that data (unless it's data
                # we made ourselves)

        # (#) Labeled OR Unlabeled: This tells us whether the file has been labeled yet.
            # An unlabeled file will only have the Freq, UserType, and UserID specifiers
            # These file readers are not equipped to detect whether a file is Labeled or
                # Unlabeled and instead treats all as Labeled

    CLASSIFICATION_TYPE_FS = "Type"
    WITH_CLASS_NUMBER_FS = "WithClassNum"
    FREQUENCE_FS = "Freq"
    BUFFER_TYPE_FS = "BufferType"
    BUFFER_NUMBER_FS = "BufferNum"
    USER_TYPE_FS = "UserType"
    USER_ID_FS = "UserID"
    LABELED_FS = "Labeled"
    UNLABELED_FS = "Unlabeled"
    # tuple to reduce dynamic changes possible
    LABELED_FILE_SPECIFIERS = (
        CLASSIFICATION_TYPE_FS,
        WITH_CLASS_NUMBER_FS,
        FREQUENCE_FS,
        BUFFER_TYPE_FS,
        BUFFER_NUMBER_FS,
        USER_TYPE_FS,
        USER_ID_FS,
        LABELED_FS # ,
        # UNLABELED_FS
    )
    UNLABELED_FILE_SPECIFIERS = (
        FREQUENCE_FS,
        USER_TYPE_FS,
        USER_ID_FS,
        # LABELED_FS,
        UNLABELED_FS
    )
    # print(len(FILE_SPECIFIERS))




    # 2. Short Descriptions *****************************************************************************************************************************



    INPUT_NAMES = {
        1: ("x", "y", "z")
    }


    # BEGINNING_DESCRIPTORS and ENDING_DESCRIPTORS will be used in file names.
    # The "<School Year>" ending descriptor is supposed to later be dynamically replaced by something like "23-24", and
        # "<Semester>" is supposed to later be dynamically replaced by either "Fall", "Spring", or "Summer".
        # If it is across multiple school years, use the start and end years.
            # For example, if it was from 23-24 to 26-27, use "23-27".
        # If it is across multiple semesters, just don't include the semester ending descriptor


    # BEGINNING_DESCRIPTORS = np.array(["COMBINED"])
    # ENDING_DESCRIPTORS = np.array(["Motion-Sessions", "<School Year>", "<Semester>"])


    NUM_OF_LABEL_TYPES = 8
    NUM_OF_BUFFER_TYPES = 1

    NUM_OF_INPUTS_PER_TYPE = {
        # if any of these are ever not 3, switch code will need to be different
            # between two different numbers of inputs
        1: 3,
        2: 3,
        3: 3,
        4: 3,
        5: 3,
        6: 3,
        7: 3,
        8: 3
    }
    NUM_OF_CLASSES_PER_TYPE = {
        1: 5,
        2: 10,
        3: 16,
        4: 4,
        5: 8,
        6: 4,
        7: 3,
        8: 2
    }
    # if 1, the ouput is a one-hot vector
    NUM_OF_OUTPUTS_PER_TYPE = {
        1: 1,
        2: 1,
        3: 1,
        4: 1,
        5: 1,
        6: 1,
        7: 1,
        8: 1
    }





    # 3. Tags and Class Names *****************************************************************************************************************************



    class Tag:
        __ambiguous_tags = set()
        
        @classmethod
        def is_ambiguous(cls, tag):
            # basically the same as 'return self.__IS_AMBIGUOUS' but allows other objects to use it
            return tag in cls.__ambiguous_tags

        def __init__(self, name, is_ambiguous = False):
            self.__NAME = str(name)
            self.__IS_AMBIGUOUS = bool(is_ambiguous)
            if self.__IS_AMBIGUOUS:
                # it kept telling me that _Tag__ambiguous_tags doesn't exist in SKDescriptors.Tag
                    # even after I manually mangled the variable name... so I am using this work around with type(self)
                type(self).__ambiguous_tags.add(self)

        def __str__(self):
            return self.__NAME
        
        def __eq__(self, other):
            if isinstance(other, SKDescriptors.Tag):
                if self is other:
                    return True
                elif self.__IS_AMBIGUOUS or other.__IS_AMBIGUOUS:
                    return NotImplemented
                return False
            elif isinstance(other, SKDescriptors.ClassType):
                return False
            return self.__NAME == other

        def __hash__(self):
            return hash((self.__NAME, self.__IS_AMBIGUOUS))
        
        def __add__(self, other):
            return self.__NAME + str(other)
        
        def __radd__(self, other):
            return str(other) + self.__NAME
    
    # class AmbTag(Tag):
    #     def __init__(self, name, children = None):
    #         super().__init__(name)
    #         if children is None:
    #             children = set()
    #         self.__children = children

    #     def __eq__(self, other):
    #         if self is other:
    #             return True
    #         if isinstance(other, (SKDescriptors.AmbTag, SKDescriptors.ClassType)):
    #             return False
    #         if isinstance(other, SKDescriptors.Tag):
    #             return False
    
    
    # # these are explicitly defined to lower the chance for typed mistakes
        # while also allowing descriptive use
    EXERCISE_TAG = Tag("Exercise")
    NONEXERCISE_TAG = Tag("Non-Exercise")
    AMBIGUOUS_EXERCISE_TAG = Tag("Ambiguous Exercise", True) # this would be used for things that may or may not be part of an exercise (certain stationary classes)
    FULL_TAG = Tag("Full")
    BORDER_TAG = Tag("Border") # Start or End -- different to "Boundary" if used
    START_TAG = Tag("Start")
    END_TAG = Tag("End")
    FORWARD_LEAN_TAG = Tag("Forward Lean")
    FORWARD_KNEE_LEAN_TAG = Tag("Forward Knee Lean")
    FORWARD_TABLE_LEAN_TAG = Tag("Forward Table Lean")
    LATERAL_LEAN_TAG = Tag("Lateral Lean")
    LATERAL_PUSH_LEAN_TAG = Tag("Lateral Push Lean")
    LATERAL_HOLD_LEAN_TAG = Tag("Lateral Hold Lean")
    LEFT_LEAN_TAG = Tag("Left Lean")
    RIGHT_LEAN_TAG = Tag("Right Lean")
    PUSHUP_TAG = Tag("Pushup")
    #boundary_tag = Tag("Boundary") # This might later be used for the edges of data files to note where they are glued together
    OTHER_TAG = Tag("Other")
    STATIONARY_TAG = Tag("Stationary")
    NONSTATIONARY_TAG = Tag("Non-Stationary")
    AMBIGUOUS_STATIONARY_TAG = Tag("Ambiguous Stationary", True) # "Other" and exercise classes that also contain Stationary portions would use this

    # set
    CLASS_TAGS = {
        EXERCISE_TAG,
        NONEXERCISE_TAG,
        AMBIGUOUS_EXERCISE_TAG,
        FULL_TAG,
        BORDER_TAG,
        START_TAG,
        END_TAG,
        FORWARD_LEAN_TAG,
        FORWARD_KNEE_LEAN_TAG,
        FORWARD_TABLE_LEAN_TAG,
        LATERAL_LEAN_TAG,
        LATERAL_PUSH_LEAN_TAG,
        LATERAL_HOLD_LEAN_TAG,
        LEFT_LEAN_TAG,
        RIGHT_LEAN_TAG,
        PUSHUP_TAG,
        OTHER_TAG,
        STATIONARY_TAG,
        NONSTATIONARY_TAG,
        AMBIGUOUS_STATIONARY_TAG
    }


    # jump to the tuple if you want the list of class names
    FORWARD_LEAN_CLASS              = FORWARD_LEAN_TAG                          # "Forward Lean"
    FORWARD_LEAN_START_CLASS        = FORWARD_LEAN_TAG + ' ' + START_TAG        # "Forward Lean Start"
    FORWARD_LEAN_END_CLASS          = FORWARD_LEAN_TAG + ' ' + END_TAG          # "Forward Lean End"
    FORWARD_KNEE_LEAN_START_CLASS   = FORWARD_KNEE_LEAN_TAG + ' ' + START_TAG   # "Forward Knee Lean Start"
    FORWARD_KNEE_LEAN_END_CLASS     = FORWARD_KNEE_LEAN_TAG + ' ' + END_TAG     # "Forward Knee Lean End"
    FORWARD_TABLE_LEAN_START_CLASS  = FORWARD_TABLE_LEAN_TAG + ' ' + START_TAG  # "Forward Table Lean Start"
    FORWARD_TABLE_LEAN_END_CLASS    = FORWARD_TABLE_LEAN_TAG + ' ' + END_TAG    # "Forward Table Lean End"
    LATERAL_LEAN_CLASS              = LATERAL_LEAN_TAG                          # "Lateral Lean"
    LATERAL_LEAN_START_CLASS        = LATERAL_LEAN_TAG + ' ' + START_TAG        # "Lateral Lean Start"
    LATERAL_LEAN_END_CLASS          = LATERAL_LEAN_TAG + ' ' + END_TAG          # "Lateral Lean End"
    LEFT_LEAN_CLASS                 = LEFT_LEAN_TAG                             # "Left Lean"
    LEFT_LEAN_START_CLASS           = LEFT_LEAN_TAG + ' ' + START_TAG           # "Left Lean Start"
    LEFT_LEAN_END_CLASS             = LEFT_LEAN_TAG + ' ' + END_TAG             # "Left Lean End"
    LEFT_PUSH_LEAN_START_CLASS      = "Left Push Lean Start"
    LEFT_PUSH_LEAN_END_CLASS        = "Left Push Lean End"
    LEFT_HOLD_LEAN_START_CLASS      = "Left Hold Lean Start"
    LEFT_HOLD_LEAN_END_CLASS        = "Left Hold Lean End"
    RIGHT_LEAN_CLASS                = RIGHT_LEAN_TAG                            # "Right Lean"
    RIGHT_LEAN_START_CLASS          = RIGHT_LEAN_TAG + ' ' + START_TAG          # "Right Lean Start"
    RIGHT_LEAN_END_CLASS            = RIGHT_LEAN_TAG + ' ' + END_TAG            # "Right Lean End"
    RIGHT_PUSH_LEAN_START_CLASS     = "Right Push Lean Start"
    RIGHT_PUSH_LEAN_END_CLASS       = "Right Push Lean End"
    RIGHT_HOLD_LEAN_START_CLASS     = "Right Hold Lean Start"
    RIGHT_HOLD_LEAN_END_CLASS       = "Right Hold Lean End"
    PUSHUP_CLASS                    = PUSHUP_TAG                                # "Pushup"
    PUSHUP_START_CLASS              = PUSHUP_TAG + ' ' + START_TAG              # "Pushup Start"
    PUSHUP_END_CLASS                = PUSHUP_TAG + ' ' + END_TAG                # "Pushup End"
    STATIONARY_CLASS                = STATIONARY_TAG                            # "Stationary"
    MOVING_CLASS                    = "Moving"
    OTHER_CLASS                     = OTHER_TAG                                 # "Other"

    CURRENTLY_USED_CLASS_NAMES = {
        FORWARD_LEAN_CLASS,
        FORWARD_LEAN_START_CLASS,
        FORWARD_LEAN_END_CLASS,
        FORWARD_KNEE_LEAN_START_CLASS,
        FORWARD_KNEE_LEAN_END_CLASS,
        FORWARD_TABLE_LEAN_START_CLASS,
        FORWARD_TABLE_LEAN_END_CLASS,
        LATERAL_LEAN_CLASS,
        LATERAL_LEAN_START_CLASS,
        LATERAL_LEAN_END_CLASS,
        LEFT_LEAN_CLASS,
        LEFT_LEAN_START_CLASS,
        LEFT_LEAN_END_CLASS,
        LEFT_PUSH_LEAN_START_CLASS,
        LEFT_PUSH_LEAN_END_CLASS,
        LEFT_HOLD_LEAN_START_CLASS,
        LEFT_HOLD_LEAN_END_CLASS,
        RIGHT_LEAN_CLASS,
        RIGHT_LEAN_START_CLASS,
        RIGHT_LEAN_END_CLASS,
        RIGHT_PUSH_LEAN_START_CLASS,
        RIGHT_PUSH_LEAN_END_CLASS,
        RIGHT_HOLD_LEAN_START_CLASS,
        RIGHT_HOLD_LEAN_END_CLASS,
        PUSHUP_CLASS,
        PUSHUP_START_CLASS,
        PUSHUP_END_CLASS,
        STATIONARY_CLASS,
        MOVING_CLASS,
        OTHER_CLASS
    }





    # 4. ClassType Definitions *****************************************************************************************************************************



    # class-tag sets
    class ClassType:
        def __init__(self, classname = "", tagset = None, inherited_types = None):
            self.__CLASSNAME = str(classname)
            if tagset is None:
                self.__TAGS = set()
            else:
                self.__TAGS = set(tagset)
            if inherited_types is None:
                inherited_types = set()
            for d in inherited_types:
                self.__TAGS.update(d) # d.tagset

        def __len__(self):
            return len(self.__TAGS)

        def __iter__(self):
            return iter(self.__TAGS)

        def __contains__(self, item):
            if SKDescriptors.Tag.is_ambiguous(item):
                return True
            return item in self.__TAGS

        def __str__(self):
            return self.__CLASSNAME
        
        def __eq__(self, other):
            if isinstance(other, SKDescriptors.ClassType):
                return self.__CLASSNAME == other.__CLASSNAME and self.__TAGS == other.__TAGS
            elif isinstance(other, SKDescriptors.Tag):
                return False
            elif isinstance(other, str):
                return self.__CLASSNAME == other
            elif isinstance(other, Iterable):
                return self.__TAGS == set(other)
            return NotImplemented

        def __hash__(self):
            return hash((self.__CLASSNAME, *self.__TAGS))


    # ABS is abstract... DO NOT USE except to build others
    ABS_EXERCISE_CTS                = ClassType("ABS " + EXERCISE_TAG,
                                                [EXERCISE_TAG])
    ABS_OTHER_CTS                   = ClassType("ABS " + OTHER_TAG,
                                                [NONEXERCISE_TAG, OTHER_TAG])

    ABS_EXERCISE_FULL_CTS           = ClassType("ABS " + EXERCISE_TAG + " " + FULL_TAG,
                                                [FULL_TAG, AMBIGUOUS_STATIONARY_TAG],
                                                [ABS_EXERCISE_CTS])


    EXERCISE_BORDER_CTS             = ClassType(EXERCISE_TAG + " " + BORDER_TAG,
                                                [BORDER_TAG, NONSTATIONARY_TAG],
                                                [ABS_EXERCISE_CTS])
    EXERCISE_START_CTS              = ClassType(EXERCISE_TAG + " " + START_TAG,
                                                [START_TAG],
                                                [EXERCISE_BORDER_CTS])
    EXERCISE_END_CTS                = ClassType(EXERCISE_TAG + " " + END_TAG,
                                                [END_TAG],
                                                [EXERCISE_BORDER_CTS])


    ABS_FORWARD_LEAN_CTS            = ClassType("ABS " + FORWARD_LEAN_CLASS,
                                                [FORWARD_LEAN_TAG],
                                                [ABS_EXERCISE_CTS])
    ABS_LATERAL_LEAN_CTS            = ClassType("ABS " + LATERAL_LEAN_CLASS,
                                                [LATERAL_LEAN_TAG],
                                                [ABS_EXERCISE_CTS])
    ABS_LEFT_LEAN_CTS               = ClassType("ABS " + LEFT_LEAN_CLASS,
                                                [LEFT_LEAN_TAG],
                                                [ABS_LATERAL_LEAN_CTS])
    ABS_RIGHT_LEAN_CTS              = ClassType("ABS " + RIGHT_LEAN_CLASS,
                                                [RIGHT_LEAN_TAG],
                                                [ABS_LATERAL_LEAN_CTS])
    ABS_PUSHUP_CTS                  = ClassType("ABS " + PUSHUP_CLASS,
                                                [PUSHUP_TAG],
                                                [ABS_EXERCISE_CTS])


    ABS_FORWARD_KNEE_LEAN_CTS       = ClassType("ABS Forward Knee Lean",
                                                [FORWARD_KNEE_LEAN_TAG],
                                                [ABS_FORWARD_LEAN_CTS])
    ABS_FORWARD_TABLE_LEAN_CTS      = ClassType("ABS Forward Table Lean",
                                                [FORWARD_TABLE_LEAN_TAG],
                                                [ABS_FORWARD_LEAN_CTS])
    ABS_LATERAL_PUSH_LEAN_CTS       = ClassType("ABS Lateral Push Lean",
                                                [LATERAL_PUSH_LEAN_TAG],
                                                [ABS_LATERAL_LEAN_CTS])
    ABS_LATERAL_HOLD_LEAN_CTS       = ClassType("ABS Lateral Hold Lean",
                                                [LATERAL_HOLD_LEAN_TAG],
                                                [ABS_LATERAL_LEAN_CTS])


    FORWARD_LEAN_FULL_CTS           = ClassType(FORWARD_LEAN_CLASS, [],
                                                [ABS_FORWARD_LEAN_CTS, ABS_EXERCISE_FULL_CTS])
    FORWARD_LEAN_START_CTS          = ClassType(FORWARD_LEAN_START_CLASS, [],
                                                [ABS_FORWARD_LEAN_CTS, EXERCISE_START_CTS])
    FORWARD_LEAN_END_CTS            = ClassType(FORWARD_LEAN_END_CLASS, [],
                                                [ABS_FORWARD_LEAN_CTS, EXERCISE_END_CTS])

    FORWARD_KNEE_LEAN_START_CTS     = ClassType(FORWARD_KNEE_LEAN_START_CLASS, [],
                                                [FORWARD_LEAN_START_CTS, ABS_FORWARD_KNEE_LEAN_CTS])
    FORWARD_KNEE_LEAN_END_CTS       = ClassType(FORWARD_KNEE_LEAN_END_CLASS, [],
                                                [FORWARD_LEAN_END_CTS, ABS_FORWARD_KNEE_LEAN_CTS])
    FORWARD_TABLE_LEAN_START_CTS    = ClassType(FORWARD_TABLE_LEAN_START_CLASS, [],
                                                [FORWARD_LEAN_START_CTS, ABS_FORWARD_TABLE_LEAN_CTS])
    FORWARD_TABLE_LEAN_END_CTS      = ClassType(FORWARD_TABLE_LEAN_END_CLASS, [],
                                                [FORWARD_LEAN_END_CTS, ABS_FORWARD_TABLE_LEAN_CTS])


    LATERAL_LEAN_FULL_CTS           = ClassType(LATERAL_LEAN_CLASS, [],
                                                [ABS_LATERAL_LEAN_CTS, ABS_EXERCISE_FULL_CTS])
    LATERAL_LEAN_START_CTS          = ClassType(LATERAL_LEAN_START_CLASS, [],
                                                [ABS_LATERAL_LEAN_CTS, EXERCISE_START_CTS])
    LATERAL_LEAN_END_CTS            = ClassType(LATERAL_LEAN_END_CLASS, [],
                                                [ABS_LATERAL_LEAN_CTS, EXERCISE_END_CTS])


    LEFT_LEAN_FULL_CTS              = ClassType(LEFT_LEAN_CLASS, [],
                                                [ABS_LEFT_LEAN_CTS, LATERAL_LEAN_FULL_CTS])
    LEFT_LEAN_START_CTS             = ClassType(LEFT_LEAN_START_CLASS, [],
                                                [ABS_LEFT_LEAN_CTS, LATERAL_LEAN_START_CTS])
    LEFT_LEAN_END_CTS               = ClassType(LEFT_LEAN_END_CLASS, [],
                                                [ABS_LEFT_LEAN_CTS, LATERAL_LEAN_END_CTS])

    LEFT_PUSH_LEAN_START_CTS        = ClassType(LEFT_PUSH_LEAN_START_CLASS, [],
                                                [ABS_LATERAL_PUSH_LEAN_CTS, LEFT_LEAN_START_CTS])
    LEFT_PUSH_LEAN_END_CTS          = ClassType(LEFT_PUSH_LEAN_END_CLASS, [],
                                                [ABS_LATERAL_PUSH_LEAN_CTS, LEFT_LEAN_END_CTS])
    LEFT_HOLD_LEAN_START_CTS        = ClassType(LEFT_HOLD_LEAN_START_CLASS, [],
                                                [ABS_LATERAL_HOLD_LEAN_CTS, LEFT_LEAN_START_CTS])
    LEFT_HOLD_LEAN_END_CTS          = ClassType(LEFT_HOLD_LEAN_END_CLASS, [],
                                                [ABS_LATERAL_HOLD_LEAN_CTS, LEFT_LEAN_END_CTS])


    RIGHT_LEAN_FULL_CTS             = ClassType(RIGHT_LEAN_CLASS, [],
                                                [ABS_RIGHT_LEAN_CTS, LATERAL_LEAN_FULL_CTS])
    RIGHT_LEAN_START_CTS            = ClassType(RIGHT_LEAN_START_CLASS, [],
                                                [ABS_RIGHT_LEAN_CTS, LATERAL_LEAN_START_CTS])
    RIGHT_LEAN_END_CTS              = ClassType(RIGHT_LEAN_END_CLASS, [],
                                                [ABS_RIGHT_LEAN_CTS, LATERAL_LEAN_END_CTS])

    RIGHT_PUSH_LEAN_START_CTS       = ClassType(RIGHT_PUSH_LEAN_START_CLASS, [],
                                                [ABS_LATERAL_PUSH_LEAN_CTS, RIGHT_LEAN_START_CTS])
    RIGHT_PUSH_LEAN_END_CTS         = ClassType(RIGHT_PUSH_LEAN_END_CLASS, [],
                                                [ABS_LATERAL_PUSH_LEAN_CTS, RIGHT_LEAN_END_CTS])
    RIGHT_HOLD_LEAN_START_CTS       = ClassType(RIGHT_HOLD_LEAN_START_CLASS, [],
                                                [ABS_LATERAL_HOLD_LEAN_CTS, RIGHT_LEAN_START_CTS])
    RIGHT_HOLD_LEAN_END_CTS         = ClassType(RIGHT_HOLD_LEAN_END_CLASS, [],
                                                [ABS_LATERAL_HOLD_LEAN_CTS, RIGHT_LEAN_END_CTS])


    PUSHUP_FULL_CTS                 = ClassType(PUSHUP_CLASS, [],
                                                [ABS_PUSHUP_CTS, ABS_EXERCISE_FULL_CTS])
    PUSHUP_START_CTS                = ClassType(PUSHUP_START_CLASS, [],
                                                [ABS_PUSHUP_CTS, EXERCISE_START_CTS])
    PUSHUP_END_CTS                  = ClassType(PUSHUP_END_CLASS, [],
                                                [ABS_PUSHUP_CTS, EXERCISE_END_CTS])


    STATIONARY_CTS                  = ClassType(STATIONARY_CLASS,
                                                [AMBIGUOUS_EXERCISE_TAG, STATIONARY_TAG])
    NONSTATIONARY_CTS = MOVING_CTS  = ClassType(MOVING_CLASS,
                                                [AMBIGUOUS_EXERCISE_TAG, NONSTATIONARY_TAG])
    OTHER_AMBSTAT_CTS               = ClassType(OTHER_CLASS,
                                                [AMBIGUOUS_STATIONARY_TAG],
                                                [ABS_OTHER_CTS])
    OTHER_NONSTAT_CTS               = ClassType(OTHER_CLASS,
                                                [NONSTATIONARY_TAG],
                                                [ABS_OTHER_CTS])





    # 5. LabelTypes' ClassTypes *****************************************************************************************************************************



    CTS_PER_TYPE = {
        1: (
            FORWARD_LEAN_FULL_CTS,          # "Forward Lean"
            LEFT_LEAN_FULL_CTS,             # "Left Lean"
            RIGHT_LEAN_FULL_CTS,            # "Right Lean"
            PUSHUP_FULL_CTS,                # "Pushup"
            OTHER_AMBSTAT_CTS               # "Other"
        ),
        2: (
            FORWARD_LEAN_START_CTS,         # "Forward Lean Start"
            FORWARD_LEAN_END_CTS,           # "Forward Lean End"
            LEFT_LEAN_START_CTS,            # "Left Lean Start"
            LEFT_LEAN_END_CTS,              # "Left Lean End"
            RIGHT_LEAN_START_CTS,           # "Right Lean Start"
            RIGHT_LEAN_END_CTS,             # "Right Lean End"
            PUSHUP_START_CTS,               # "Pushup Start"
            PUSHUP_END_CTS,                 # "Pushup End"
            STATIONARY_CTS,                 # "Stationary"
            OTHER_NONSTAT_CTS               # "Other"
        ),
        3: (
            FORWARD_KNEE_LEAN_START_CTS,    # "Forward Knee Lean Start"
            FORWARD_KNEE_LEAN_END_CTS,      # "Forward Knee Lean End"
            FORWARD_TABLE_LEAN_START_CTS,   # "Forward Table Lean Start"
            FORWARD_TABLE_LEAN_END_CTS,     # "Forward Table Lean End"
            LEFT_PUSH_LEAN_START_CTS,       # "Left Push Lean Start"
            LEFT_PUSH_LEAN_END_CTS,         # "Left Push Lean End"
            LEFT_HOLD_LEAN_START_CTS,       # "Left Hold Lean Start"
            LEFT_HOLD_LEAN_END_CTS,         # "Left Hold Lean End"
            RIGHT_PUSH_LEAN_START_CTS,      # "Right Push Lean Start"
            RIGHT_PUSH_LEAN_END_CTS,        # "Right Push Lean End"
            RIGHT_HOLD_LEAN_START_CTS,      # "Right Hold Lean Start"
            RIGHT_HOLD_LEAN_END_CTS,        # "Right Hold Lean End"
            PUSHUP_START_CTS,               # "Pushup Start"
            PUSHUP_END_CTS,                 # "Pushup End"
            STATIONARY_CTS,                 # "Stationary"
            OTHER_NONSTAT_CTS               # "Other"
        ),
        4: (
            FORWARD_LEAN_FULL_CTS,          # "Forward Lean"
            LATERAL_LEAN_FULL_CTS,          # "Lateral Lean"
            PUSHUP_FULL_CTS,                # "Pushup"
            OTHER_AMBSTAT_CTS               # "Other"
        ),
        5: (
            FORWARD_LEAN_START_CTS,         # "Forward Lean Start"
            FORWARD_LEAN_END_CTS,           # "Forward Lean End"
            LATERAL_LEAN_START_CTS,         # "Lateral Lean Start"
            LATERAL_LEAN_END_CTS,           # "Lateral Lean End"
            PUSHUP_START_CTS,               # "Pushup Start"
            PUSHUP_END_CTS,                 # "Pushup End"
            STATIONARY_CTS,                 # "Stationary"
            OTHER_NONSTAT_CTS               # "Other"
        ),
        6: (
            EXERCISE_START_CTS,             # "Exercise Start"
            EXERCISE_END_CTS,               # "Exercise End"
            STATIONARY_CTS,                 # "Stationary"
            OTHER_NONSTAT_CTS               # "Other"
        ),
        7: (
            EXERCISE_BORDER_CTS,            # "Exercise Border"
            STATIONARY_CTS,                 # "Stationary"
            OTHER_NONSTAT_CTS               # "Other"
        ),
        8: (
            MOVING_CTS,                     # "Moving"
            STATIONARY_CTS                  # "Stationary"
        )
    }


    # each buffer type has a dictionary;
        # the keys of each dictionary have precedence over the values;
        # rules (key-value pairs) listed sooner have priority over later rules
    PRECEDENCE_OF_TAGS_PER_BUFFER_TYPE = {
        1: {
            # exercises should have precedence over "Other" and "Stationary"
            EXERCISE_TAG: (NONEXERCISE_TAG, AMBIGUOUS_EXERCISE_TAG),
            # we actually don't want the following line because then in Type 3 and Type 5,
                # Stationary would have precedence over Other (which would be incorrect)
            #AMBIGUOUS_EXERCISE_TAG: (NONEXERCISE_TAG,) # the comma here is to make it a tuple #
            # "Other" and "Moving" should have precedence over "Stationary"
            NONSTATIONARY_TAG: (STATIONARY_TAG, AMBIGUOUS_STATIONARY_TAG)
        }
    }




    # 6. Supporting Functions *****************************************************************************************************************************



    def find_subclasses(classtype: "SKDescriptors.ClassType", classification: int):
        SKDescriptors.validate_class_type(classification)
        ret_tuples = []
        for i, subclasstype in enumerate(SKDescriptors.CTS_PER_TYPE[classification]):
            found = True
            for tag in classtype:
                if tag not in subclasstype:
                    found = False
                    break
            if found:
                ret_tuples += [(i, subclasstype)]
        return ret_tuples
    


    @cache_with_except
    def get_superclass_dict(input_label_type, output_label_type):
        # rules (with subclass from input and superclass from output):
            # 1.  each superclass must have  at least one   subclass (Onto / Surjective)
            # 2.  each   subclass must have  at  most one superclass (Functional)
            # 3.  each   subclass must have  at least one superclass (Total / Serial)
            # BUT each superclass  CAN have more than one   subclass (Not Necessarily One-to-One / Injective)
        SKDescriptors.validate_class_type(input_label_type)
        SKDescriptors.validate_class_type(output_label_type)
        superclasses = {}

        # superclasses = np.empty(len(SKDescriptors.CTS_PER_TYPE[input_label_type]))
        # superclasses.fill(np.nan)
        for i, cts in enumerate(SKDescriptors.CTS_PER_TYPE[output_label_type]):
            temp = SKDescriptors.find_subclasses(cts, input_label_type)
            
            # rule 1.
            if not temp:
                raise AssertionError(f"Um, there doesn't appear to be any subclasses for classification type {output_label_type}'s '{cts}' class in classification type {input_label_type}... superclass dict so far:\n{superclasses}")
            
            for j, subcts in temp:

                # rule 2.
                if superclasses.get(j, None) is not None:
                # if superclasses[j] != np.nan:
                    raise AssertionError(f"Class {j} from classification type {input_label_type} is a subclass to more than one class in classification type {output_label_type}: class {superclasses[j]} and class {i}")
                
                superclasses[j] = i

        # rule 3.
        for j, subcts in enumerate(SKDescriptors.CTS_PER_TYPE[input_label_type]):
            if superclasses.get(j, None) is None:
            # if superclasses[j] == np.nan:
                raise AssertionError(f"Class {j} from classification type {input_label_type} is not a subclass to any class in classification type {output_label_type}... superclass dict:\n{superclasses}")
        
        return superclasses



    @cache_with_except
    def get_subclass_dict(input_label_type, output_label_type):
        return invert(SKDescriptors.get_superclass_dict(input_label_type, output_label_type))
        # superclasses = SKDescriptors.get_superclass_dict(input_label_type, output_label_type)
        # subclasses = {}
        # for sub, super in superclasses.items():
        #     subclasses.setdefault(super, [])
        #     subclasses[super].append(sub)
        # return subclasses
    


    def format_classes(classification_type = None, target_classes = None, format = 'str'):
        formatted_classes = []
        if classification_type is None or target_classes is None:
            return formatted_classes
        # if tc_info[1] != 1:
        target_classes = list(iterable_info(target_classes)[0])
        if not target_classes:
            return formatted_classes

        match format:
            case 'str' | 'cts':
                for i, cts in enumerate(SKDescriptors.CTS_PER_TYPE[classification_type]):
                    if cts in target_classes:
                        formatted_classes.append(i)


                if len(formatted_classes) != len(target_classes):
                    print(f"\nWARNING: The following target_classes were not kept as they do not correspond to classification_type '{classification_type}':")
                    for cts in target_classes:
                        if not any(cts == SKDescriptors.CTS_PER_TYPE[classification_type][i] for i in formatted_classes):
                            print(f"\t{cts}")


            case 'tag':
                for i, cts in enumerate(SKDescriptors.CTS_PER_TYPE[classification_type]):
                    for tag in target_classes:
                        if tag in cts:
                            formatted_classes.append(i)


            case 'num' | 'int':
                forwarn_active = False
                for i in target_classes:
                    if i in range(SKDescriptors.NUM_OF_CLASSES_PER_TYPE[classification_type]):
                        formatted_classes.append(i)
                        continue

                    if not forwarn_active:
                        print(f"\nWARNING: The following target_classes were not kept as they do not correspond to classification_type '{classification_type}':")
                        forwarn_active = True
                    print(f"\t{i}")
                if forwarn_active:
                    print()
                

            case _ :
                raise AssertionError("You can only pass 'str', 'tag', 'cts', 'num', or 'int' into the format parameter of JFSKLoader.format_classes()\
                                     \n\t\t'str' is used when you want to pass the class names (as strings) of classes it should repress\
                                     \n\t\t'tag' is used when you want to pass the class tags (as strings) it should repress\
                                     \n\t\t'cts' is used when you want to pass the classes (as ClassTypes) it should repress\
                                     \n\t\t'int' and 'num' are used when you want to pass the indices of the classes it should repress")
            
            
        return formatted_classes





    # 7. Validating Functions *****************************************************************************************************************************



    def validate_all_descriptors():
        # validating type values' and type dictionaries' consistency
        # all label type dictionaries have the same length
        can_use_class_dictionaries = all(SKDescriptors.NUM_OF_LABEL_TYPES == n for n in (len(SKDescriptors.NUM_OF_INPUTS_PER_TYPE), len(SKDescriptors.NUM_OF_CLASSES_PER_TYPE), len(SKDescriptors.NUM_OF_OUTPUTS_PER_TYPE), len(SKDescriptors.CTS_PER_TYPE)))
        # can_use_classtag_dictionary = all(SKDescriptors.NUM_OF_CLASSES_PER_TYPE == len(c) for c in SKDescriptors.LIST_OF_CLASSES_AND_TAGS_PER_TYPE.values())
        can_use_classtag_dictionary = True
        for classnum, classtuple in zip(SKDescriptors.NUM_OF_CLASSES_PER_TYPE.values(), SKDescriptors.CTS_PER_TYPE.values()):
            can_use_classtag_dictionary &= (classnum == len(classtuple))
        can_use_buffer_dictionary = (SKDescriptors.NUM_OF_BUFFER_TYPES == len(SKDescriptors.PRECEDENCE_OF_TAGS_PER_BUFFER_TYPE))
        # NOTE: this line is not required, but it is there to catch when this changes
            # if you have changed stuff that causes this line to fail, that means there are lots of other places it may fail as well
        input_is_three = (len(n) == 3 for n in SKDescriptors.INPUT_NAMES.values()) and all(n == 3 for n in SKDescriptors.NUM_OF_INPUTS_PER_TYPE.values())
        # DO NOT comment the below assert line or use makeshift bypass code UNLESS you have a backup of a version that actually runs;
            # preferably, you would instead work out some additional logic that takes into account your case
        if not (can_use_class_dictionaries and can_use_classtag_dictionary and can_use_buffer_dictionary):
            raise AssertionError("Descriptor dictionary lengths do not match.")
        elif not (input_is_three):
            raise AssertionError("Woah buddy, it looks like you have a lot of reworking stuff to do before you can use anything but x, y, and z for inputs...\
                                 \n\tStart with adding 'InputType' to the File Specifiers list \
                                 \n\tThen you'll have to rework stuff that requires or assumes that the first element is always there or always 'Type'\
                                 \n\t\tFor instance, the file name reader toward the bottom of HelperFunctions.ipynb\
                                 \n\tConsider also renaming 'Type' to 'LabelType' (or something similar) to keep things from being more confusing\
                                 \n\t\tFor this, you may want to make something that the rest of the code can check for old values in\
                                 \n\t\t\t\tsince files may still have 'Type', so this would allow backwards compatibility")

    # you should pass False to this function only if the code using it properly handles an invalid classtype and/or classnum
    def validate_class_type(classtype, classnum = 0, assertion = True):
        validity = True
        # is classtype in the valid range?
        if classtype <= 0 or classtype > SKDescriptors.NUM_OF_LABEL_TYPES:
            validity = False
        # if classnum is provided, does it correspond with classtype?
        if classnum != 0 and classnum != SKDescriptors.NUM_OF_CLASSES_PER_TYPE.get(classtype, -1):
            validity = False

        if assertion and not validity:
            raise AssertionError(f"classtype {classtype} is invalid or does not correspond with classnum {classnum}.")
        return validity

    # you should pass False to this function only if the code using it properly handles an invalid buffertype and/or buffernum
    def validate_buffer_type(buffertype, buffernum = 0, assertion = True):
        validity = True
        # is buffertype in the valid range?
        if buffertype <= 0 or buffertype > SKDescriptors.NUM_OF_BUFFER_TYPES:
            validity = False
        # if buffernum is provided, is it nonnegative?
        if buffernum < 0:
            validity = False

        if assertion and not validity:
            raise AssertionError(f"buffertype {buffertype} or buffernum {buffernum} is invalid.")
        return validity

# NEVER comment the `validate_all_descriptors()` below. NO exceptions.
    # if you really want to use some makeshift bypass code, go do so elsewhere
SKDescriptors.validate_all_descriptors()

In [ ]:
class SKFileNameHandler:

    def get_filename_for_classtype(class_type):
        with open('datadirs.json') as f:
            d = jsonload(f)
            return d['datapath'] + d[f'type{class_type}']
        # raise NotImplementedError(f"There is no datafile for class type {class_type}")


    # this returns:
        # the input file's directory (where it is in the computer),
        # the beginning descriptors (the file descriptors that come before the file specifiers),
        # the file name (commented out code corrected it if it was missing any file specifiers,
            # but this is functionality that could be -- and has been -- replaced by build_file_name)
            # (the file name includes everything but the directory),
        # the ending descriptors (the file descriptors that come after the file specifiers),
        # the file's extension (the file type), and
        # the values for all file specifiers (as a dict)
            # (this does not yet include the "Labeled" or "Unlabeled" file specifiers)
    def read_data_file_name(file_path, feedback = True):
        specifier_values = {} #np.zeros((len(Converter.FILE_SPECIFIERS), 1))
        beginning_descriptors = []
        ending_descriptors = []
        # at the start of each iteration (except the first),
            # separator_index points to the dash just before the file specifier;
            # by the end of each iteration, it points to the next dash
        # rfind() finds the right-most instance;
            # using separator_index, we separate the directory and the file name
        separator_index = file_path.rfind('/')
        file_directory = file_path[ : (separator_index + 1)]
        # we initialize output_file_name like this in case it has some
            # descriptors at the start of the file, before any file specifiers
        temp_index = file_path.find(SKDescriptors.LABELED_FILE_SPECIFIERS[0])
        file_name = file_path[(separator_index + 1) : ] # used to be [(separator_index + 1) : temp_index]
        #print(file_name)


        # while we haven't reached the beginning of the file specifier list
        while (separator_index + 1) < temp_index:
            # separator_index_2 points to just before each descriptor,
                # and separator_index points to just after
            separator_index_2 = separator_index
            separator_index = file_path.find('_', separator_index)
            beginning_descriptors.append(file_path[(separator_index_2 + 1) : separator_index])


        for fs in SKDescriptors.LABELED_FILE_SPECIFIERS:
            if fs == SKDescriptors.LABELED_FS:
                break
            # ignore this comment
            # if(i != 0): (since we already do this for the first iteration beforehand)
            if fs != SKDescriptors.LABELED_FILE_SPECIFIERS[0]:
                temp_index = file_path.find(fs, separator_index)

            # if temp_index is a substring of input_file_name
            if temp_index >= 0:
                # temp_index is the index of the start of the number value for the specifier
                    # (we treat this as unrelated to dash_index in case the value has more than one digit)
                temp_index += len(fs)
                # we are guaranteed to have a dash after each file specifier;
                    # this includes the last one since "Labeled" should be a final
                    # file specifier with no value for every labeled data set.
                    # "Unlabeled" may later be used, but code needs to change
                separator_index = file_path.find('-', temp_index)
                # this makes sure we get the full number
                specifier_values[fs] = (int)(file_path[temp_index : separator_index])
            else: #values[i] used to automatically stay 0
                # WithClassNum may not be included in older files,
                    # but this is not supposed to raise exceptions,
                    # so we correct it since we have the necessary information
                if fs == SKDescriptors.WITH_CLASS_NUMBER_FS:
                    specifier_values[fs] = SKDescriptors.NUM_OF_CLASSES_PER_TYPE[specifier_values[SKDescriptors.CLASSIFICATION_TYPE_FS]]
                else:
                    specifier_values[fs] = 0

            #file_name += f'{fs}{specifier_values[fs]}-'

        # THIS IS ONLY HERE BECAUSE WE ARE ASSUMING THE FILE IS LABELED;
            # IF YOU ARE NOW PROCESSING UNLABELED FILES, ADJUST THIS
        #file_name += 'Labeled'

        SKDescriptors.validate_class_type(specifier_values[SKDescriptors.CLASSIFICATION_TYPE_FS], specifier_values[SKDescriptors.WITH_CLASS_NUMBER_FS])
        # if buffertype != 0
        if specifier_values[SKDescriptors.BUFFER_TYPE_FS]:
            SKDescriptors.validate_buffer_type(specifier_values[SKDescriptors.BUFFER_TYPE_FS], specifier_values[SKDescriptors.BUFFER_NUMBER_FS])

        # this line makes sure we keep track of the end of the file specifier section
        temp_index = separator_index + len(SKDescriptors.LABELED_FS)
        # if there are no ending separators, separator_index_2 will be -1
            # (I don't know what separator_index will be)
        # if there is one ending separator, only separator_index will be -1.
            # the next step will not change anything
        # when we reach the last one normally, the separator_index will be -1.
            # the next step will not change anything
        # these two lines already prepare the first segment
        separator_index_2 = file_path.find('_', separator_index)
        separator_index = file_path.find('_', separator_index_2 + 1)
        # if it found an underscore indicating ending descriptors
        if(separator_index_2 > 0):
            # we purposefully decide to flip the order of operation versus incrementation
                # here as compared to the order of the beginning_descriptors reader
            # while we haven't reached the extension of file_path
            while file_path[separator_index_2] != '.':
                # here, if we have found the last ending_descriptor,
                    # the find function will not have found another
                    # underscore, and will return -1 for separator_index,
                    # but we do not want to include the extension
                    # (.csv or the like)
                if separator_index < 0:
                    separator_index = file_path.find('.', separator_index_2)
                    if separator_index < 0:
                        raise ValueError(f'"{file_path}" has no "." to mark the extension')

                ending_descriptors.append(file_path[(separator_index_2 + 1) : separator_index])
                # separator_index_2 points to just before each descriptor,
                    # and separator_index points to just after
                separator_index_2 = separator_index
                separator_index = file_path.find('_', separator_index_2 + 1)
        else:
            temp_index = file_path.find('.', temp_index)


        file_extension = file_path[separator_index_2 : ]


        file_info = {
            "File Directory": file_directory,
            "Beginning Descriptors": beginning_descriptors,
            "File Name": file_name,
            "Ending Descriptors": ending_descriptors,
            "File Extension": file_extension
        }

        #file_name += file_path[(temp_index + 1) : ]
        if feedback:
            print("Reading file...")
            print(f'\tFile directory from cwd is "{file_directory}"')
            print(f'\tFile name is "{file_name}".')
            print(f'\tFile name read as "{SKFileNameHandler.build_data_file_name(specifier_values, beginning_descriptors, ending_descriptors, file_extension)}".')
        return file_info, specifier_values



    # beginning_descriptors and ending_descriptors do not include the separating underscores;
        # ending_descriptors do not include the value-less file specifier "Labeled";
        # both should be numpy ARRAYS (though they can be empty)
    def build_data_file_name(file_specifier_values, beginning_descriptors = [], ending_descriptors = [], file_extension = '.csv'):
        # this ensures the file name is not empty so we can add to it
        file_name = ''

        for bd in beginning_descriptors:
            file_name += bd + '_'

        # this set of lines will need to be changed if we ever want to use this function
            # on Unlabeled files to convert their data
        for fs in file_specifier_values:
            file_name += f'{fs}{file_specifier_values[fs]}-'
        file_name += SKDescriptors.LABELED_FS

        for ed in ending_descriptors:
            file_name += '_' + ed

        file_name += file_extension

        return file_name



    def read_file_extension(file_name):
        ext_index = file_name.rfind('.')
        if ext_index == -1:
            extension = ''
        else:
            extension = file_name[ext_index : ]
        return extension